<a href="https://colab.research.google.com/github/MBrandao07/Previsao_Demandas_Market_Show/blob/main/Codigos/5_Projeto_Previs%C3%A3o_Demanda_Segmenta%C3%A7%C3%A3o_Modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6- Modelagem - Criando modelos segmentados

## Funções

In [ ]:
# Função para gerar os metadados do dataframe
def generate_metadata(dataframe):
    """
    Gera um dataframe contendo metadados das colunas do dataframe fornecido.

    :param dataframe: DataFrame para o qual os metadados serão gerados.
    :return: DataFrame contendo metadados.
    """

    # Coleta de metadados básicos
    metadata = pd.DataFrame({
        'nome_variavel': dataframe.columns,
        'tipo': dataframe.dtypes,
        'qt_nulos': dataframe.isnull().sum(),
        'percent_nulos': round((dataframe.isnull().sum() / len(dataframe))* 100,2),
        'cardinalidade': dataframe.nunique(),
    })
    metadata=metadata.sort_values(by='percent_nulos',ascending=False)
    metadata = metadata.reset_index(drop=True)

    return metadata

### Instalando as bibliotecas necessárias

In [ ]:
!pip install catboost
!pip install optuna

### Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import time
import lightgbm as lgb
import catboost as cb
import joblib
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import optuna
from optuna.samplers import TPESampler
from optuna.study import MaxTrialsCallback
from optuna.trial import TrialState

# Suprimir os alertas do LightGBM
warnings.filterwarnings('ignore', category=UserWarning, module='lightgbm')

### Conectando ao drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Importando as bases de dados

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/train_data_fs.csv')
val = pd.read_csv('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/val_data_fs.csv')

In [ ]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type_stores,...,rolling_mean_180,rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,650430,2014-01-02,1,AUTOMOTIVE,3.0,0,95.14,Quito,Pichincha,D,...,5.805908,5.971493,2014,1,2,3,False,285.42,0.031532,31.0
1,652212,2014-01-03,1,AUTOMOTIVE,0.0,0,93.66,Quito,Pichincha,D,...,5.805908,5.971493,2014,1,3,4,False,0.00,0.000000,35.0
2,653994,2014-01-04,1,AUTOMOTIVE,0.0,0,93.12,Quito,Pichincha,D,...,5.805908,5.971493,2014,1,4,5,True,558.72,0.064433,35.0
3,655776,2014-01-05,1,AUTOMOTIVE,1.0,0,93.12,Quito,Pichincha,D,...,5.805908,5.971493,2014,1,5,6,True,558.72,0.064433,36.0
4,657558,2014-01-06,1,AUTOMOTIVE,6.0,0,93.12,Quito,Pichincha,D,...,5.805908,5.971493,2014,1,6,0,False,558.72,0.064433,39.0


In [ ]:
val.head()

,id,date,store_nbr,family,sales,onpromotion,oil_price,city,state,type_stores,...,rolling_mean_180,rolling_mean_360,year,month,day,day_of_week,is_weekend,sales_oil_price_interaction_mult,sales_oil_price_interaction_div,target
0,2596374,2017-01-01,1,AUTOMOTIVE,0.0,0,52.36,Quito,Pichincha,D,...,3.961111,4.355556,2017,1,1,6,True,209.44,0.076394,45.0
1,2598156,2017-01-02,1,AUTOMOTIVE,5.0,0,52.36,Quito,Pichincha,D,...,3.955556,4.355556,2017,1,2,0,False,209.44,0.076394,42.0
2,2599938,2017-01-03,1,AUTOMOTIVE,4.0,0,52.36,Quito,Pichincha,D,...,3.966667,4.352778,2017,1,3,1,False,209.44,0.076394,41.0
3,2601720,2017-01-04,1,AUTOMOTIVE,1.0,0,53.26,Quito,Pichincha,D,...,3.955556,4.338889,2017,1,4,2,False,53.26,0.018776,42.0
4,2603502,2017-01-05,1,AUTOMOTIVE,2.0,0,53.77,Quito,Pichincha,D,...,3.916667,4.341667,2017,1,5,3,False,107.54,0.037195,42.0


In [ ]:
generate_metadata(train)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,id,int64,0,0.0,180180
1,date,object,0,0.0,1092
2,store_nbr,int64,0,0.0,5
3,family,object,0,0.0,33
4,sales,float64,0,0.0,44453
5,onpromotion,int64,0,0.0,223
6,oil_price,float64,0,0.0,676
7,city,object,0,0.0,2
8,state,object,0,0.0,2
9,type_stores,object,0,0.0,1


In [ ]:
generate_metadata(val)

,nome_variavel,tipo,qt_nulos,percent_nulos,cardinalidade
0,id,int64,0,0.0,22440
1,date,object,0,0.0,136
2,store_nbr,int64,0,0.0,5
3,family,object,0,0.0,33
4,sales,float64,0,0.0,7494
5,onpromotion,int64,0,0.0,163
6,oil_price,float64,0,0.0,83
7,city,object,0,0.0,2
8,state,object,0,0.0,2
9,type_stores,object,0,0.0,1


## Avaliando o modelo vigente

In [ ]:
# Carregar o modelo salvo
pipeline_vigente = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_model.pkl')

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
val.loc[:,categorical_features] = val[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

# Previsões no treino
train_predictions = pipeline_vigente.predict(X_train)
train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
train_mae = mean_absolute_error(y_train, train_predictions)

# Previsões no teste
test_predictions = pipeline_vigente.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))
test_mae = mean_absolute_error(y_test, test_predictions)

results = []
model_name = 'Modelo Vigente'

results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))

# Mostrar os resultados
results_df1_vigente = pd.DataFrame(results, columns=['Model', 'Train RMSE', 'Test RMSE', 'Train MAE', 'Test MAE'])

results_df1_vigente

/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator OneHotEncoder from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ColumnTransformer from version 1.3.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.3.2 when using version 1.6.1. This might lead to breaking

,Model,Train RMSE,Test RMSE,Train MAE,Test MAE
0,Modelo Vigente,11089.268817,12486.56653,3387.497013,3333.040899


## Treinando modelos segmentados por loja

## Loja 1

### Criando um modelo segmentado para a loja 1

In [ ]:
# definindo uma loja específica
store_nbr = 1

# filtrando os dados de treino e teste para a loja 1
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# definindo features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# separando as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# convertendo as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# definindo os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # armazenando as predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# avaliando os modelos para a loja 1
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# calculando métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

Treinando o algoritmo: LinearRegression para a loja 1
Treino finalizado. Tempo gasto no treinamento: 1.10 segundos
RMSE no treino: 1642.3218407068707 
RMSE no teste: 1052.6186164011192 
MAE no treino: 543.8049568409053 
MAE no teste: 483.19607960250147 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 1
Treino finalizado. Tempo gasto no treinamento: 6.49 segundos


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


RMSE no treino: 722.4813770292874 
RMSE no teste: 1080.0119565356586 
MAE no treino: 292.3798559784518 
MAE no teste: 415.3110253290999 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 1
Treino finalizado. Tempo gasto no treinamento: 9.95 segundos
RMSE no treino: 1283.9692711409464 
RMSE no teste: 1101.072954394056 
MAE no treino: 445.22286161825554 
MAE no teste: 452.3509606778282 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a loja 1
Treino finalizado. Tempo gasto no treinamento: 246.79 segundos
RMSE no treino: 1158.2967013263824 
RMSE no teste: 1060.1175171813175 
MAE no treino: 404.5547238185196 
MAE no teste: 431.58509411248326 
----------------------------------------------------------------------
Treinando o algoritmo: XGBoost para a loja 1
Treino finalizado. Tempo gasto no treinamento: 6.45 segundos
RMSE no treino: 612.6416228032565 
RMSE no 

### Comparando com o modelo único criado anteriormente

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_gradboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')
preds_gradboost = pd.DataFrame(pipeline_best_model_gradboost.predict(X_test))
preds_gradboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 1
df = pd.concat([X_test,preds_gradboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 1].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_gradboost_loja_1 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1027.3938759178006
mae = 403.2658969141571


Podemos verificar que o modelo de LightGBM segmentado da loja 1 apresentou o melhor desempenho entre todos os modelos (porém não mais do que o modelo único), então vamos tentar tunar seus hiperparâmetros para melhorar seu resultado.


### Tunando os hiperparâmetros do melhor modelo para a loja 1

In [ ]:
warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.utils.validation')

In [ ]:
# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Função objetiva para a otimização do Optuna
def objective(trial):
    # Hiperparâmetros a otimizar
    num_leaves = trial.suggest_int('num_leaves', 20, 200)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2, log=True)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    min_child_samples = trial.suggest_int('min_child_samples', 5, 50)

    # Definir o modelo LightGBM
    model = lgb.LGBMRegressor(
        num_leaves=num_leaves,
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        min_child_samples=min_child_samples,
        random_state=42,
        n_jobs=-1
    )

    # Pipeline com pré-processador e modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar
    pipeline.fit(X_train_store, y_train_store)

    # Prever
    y_pred = pipeline.predict(X_test_store)

    # Calcular MSE
    mse = mean_squared_error(y_test_store, y_pred)
    return mse

# Callback de EarlyStopping
class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Early stopping após 10 iterações sem melhoria
early_stopping = EarlyStoppingCallback(patience=10)

# Rodar Optuna
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=20, n_jobs=-1, callbacks=[early_stopping])

# Resultados
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor MSE: {study.best_value:.4f}")

[I 2025-08-21 13:05:29,588] A new study created in memory with name: no-name-ce1f67d7-b854-466b-b153-d332022c98f1
[I 2025-08-21 13:05:37,136] Trial 0 finished with value: 1172479.7874929924 and parameters: {'num_leaves': 140, 'max_depth': 3, 'learning_rate': 0.022715882063806245, 'n_estimators': 605, 'subsample': 0.8535919932423468, 'colsample_bytree': 0.7739132988416386, 'min_child_samples': 41}. Best is trial 0 with value: 1172479.7874929924.
[I 2025-08-21 13:05:44,356] Trial 2 finished with value: 1458495.6143497669 and parameters: {'num_leaves': 161, 'max_depth': 5, 'learning_rate': 0.12453173956900905, 'n_estimators': 555, 'subsample': 0.5094096369187446, 'colsample_bytree': 0.9779211660910245, 'min_child_samples': 36}. Best is trial 0 with value: 1172479.7874929924.
[I 2025-08-21 13:05:47,247] Trial 3 finished with value: 3633324.5024416246 and parameters: {'num_leaves': 142, 'max_depth': 3, 'learning_rate': 0.020002140178265126, 'n_estimators': 111, 'subsample': 0.66525489688749

Melhores hiperparâmetros: {'num_leaves': 65, 'max_depth': 8, 'learning_rate': 0.010441116727199818, 'n_estimators': 988, 'subsample': 0.5984389370518212, 'colsample_bytree': 0.8167237112906434, 'min_child_samples': 31}
Melhor MSE: 787136.4184


In [ ]:
best_params = study.best_params
# Melhores hiperparâmetros: {'num_leaves': 126, 'max_depth': 10, 'learning_rate': 0.012549672304914734, 'n_estimators': 676, 'subsample': 0.5971182240632279, 'colsample_bytree': 0.7097042049717543, 'min_child_samples': 40}

# Definir o modelo LightGBM com os melhores hiperparâmetros
model = lgb.LGBMRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    num_leaves=best_params.get('num_leaves'),
    subsample=best_params.get('subsample'),
    colsample_bytree=best_params.get('colsample_bytree'),
    min_child_samples=best_params.get('min_child_samples'),
    random_state=42,
    n_jobs=-1
)

# Pipeline com pré-processamento
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Treinamento
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja1 = pipeline.predict(X_test_store)

# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja1.pkl')

['/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja1.pkl']

In [ ]:
# Conferindo o desempenho da versão salva

results = []

# Previsões no teste
test_predictions = pipeline.predict(X_test_store)
test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
test_mae = mean_absolute_error(y_test_store, test_predictions)

print(f"RMSE no teste: {test_rmse} \n"
      f"MAE no teste: {test_mae} \n"
      f"{70*'-'}")

RMSE no teste: 887.2070887853356 
MAE no teste: 333.04729823355797 
----------------------------------------------------------------------


## Loja 2

### Criando um modelo segmentado para a loja 2



In [ ]:
# definindo uma loja específica
store_nbr = 2

# filtrando os dados de treino e teste para a loja 2
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# definindo features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# separando as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# convertendo as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# definindo os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # armazenando as predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# avaliando os modelos para a loja 2
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# calculando métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

Treinando o algoritmo: LinearRegression para a loja 2
Treino finalizado. Tempo gasto no treinamento: 0.51 segundos
RMSE no treino: 4355.733324029477 
RMSE no teste: 1571.3108155343944 
MAE no treino: 990.8583902154605 
MAE no teste: 586.2408124009623 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 2
Treino finalizado. Tempo gasto no treinamento: 3.42 segundos
RMSE no treino: 1603.0055076196518 
RMSE no teste: 11373.474673808096 
MAE no treino: 457.82912360277896 
MAE no teste: 2193.4925204622723 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 2
Treino finalizado. Tempo gasto no treinamento: 11.82 segundos
RMSE no treino: 2431.4160533184354 
RMSE no teste: 9924.478902738605 
MAE no treino: 728.3337758519627 
MAE no teste: 1848.3684440127615 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting pa

### Comparando com o modelo único criado anteriormente

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_gradboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')
preds_gradboost = pd.DataFrame(pipeline_best_model_gradboost.predict(X_test))
preds_gradboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 2
df = pd.concat([X_test,preds_gradboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 2].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_gradboost_loja_2 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1747.7095564140077
mae = 575.013895979778


Na loja 2, o modelo segmentado de Regressão Linear conseguiu o melhor desempenho, então vamos tunar os hiperparâmetros desse modelo.

### Tunando os hiperparâmetros do melhor modelo para a loja 2

In [ ]:
from sklearn.linear_model import ElasticNet
from optuna.samplers import TPESampler

# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Função objetiva para a otimização do Optuna
def objective(trial):
    # Hiperparâmetros a otimizar
    alpha = trial.suggest_float('alpha', 1e-4, 10.0, log=True)
    l1_ratio = trial.suggest_float('l1_ratio', 0.0, 1.0)  # 0 = Ridge, 1 = Lasso

    # Modelo ElasticNet (variação da regressão linear regularizada)
    model = ElasticNet(
        alpha=alpha,
        l1_ratio=l1_ratio,
        random_state=42,
        max_iter=5000
    )

    # Pipeline com pré-processador e modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar
    pipeline.fit(X_train_store, y_train_store)

    # Prever
    y_pred = pipeline.predict(X_test_store)

    # Calcular MSE
    mse = mean_squared_error(y_test_store, y_pred)
    return mse

# Callback de EarlyStopping (aqui não faz tanta diferença, mas mantive)
class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Early stopping após 10 iterações sem melhoria
early_stopping = EarlyStoppingCallback(patience=10)

# Rodar Optuna
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=30, n_jobs=-1, callbacks=[early_stopping])

# Resultados
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor MSE: {study.best_value:.4f}")

[I 2025-08-21 17:11:11,090] A new study created in memory with name: no-name-17238bde-7603-4ce5-9941-23b5f622fb3d
[I 2025-08-21 17:11:17,966] Trial 1 finished with value: 2166020.4243511446 and parameters: {'alpha': 3.6862365787246016, 'l1_ratio': 0.8470627095087423}. Best is trial 1 with value: 2166020.4243511446.
[I 2025-08-21 17:11:37,492] Trial 0 finished with value: 2629989.2047867747 and parameters: {'alpha': 0.07774147102163735, 'l1_ratio': 0.6556448207713227}. Best is trial 1 with value: 2166020.4243511446.
[I 2025-08-21 17:12:04,794] Trial 3 finished with value: 2596193.447471136 and parameters: {'alpha': 0.015222281748449003, 'l1_ratio': 0.13405042482960905}. Best is trial 1 with value: 2166020.4243511446.
/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 

Melhores hiperparâmetros: {'alpha': 3.6862365787246016, 'l1_ratio': 0.8470627095087423}
Melhor MSE: 2166020.4244


In [ ]:
best_params = study.best_params

# Define o modelo de regressão linear (ElasticNet)
model = ElasticNet(
    alpha=best_params['alpha'],
    l1_ratio=best_params.get('l1_ratio', 0.5),  # default 0.5 caso não tenha sido otimizado
    random_state=42,
    max_iter=5000
)

# Pipeline com pré-processamento
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Treinamento
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja2 = pipeline.predict(X_test_store)

# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja2.pkl')

['/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja2.pkl']

In [ ]:
# Conferindo o desempenho da versão salva

results = []

# Previsões no teste
test_predictions = pipeline.predict(X_test_store)
test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
test_mae = mean_absolute_error(y_test_store, test_predictions)

print(f"RMSE no teste: {test_rmse} \n"
      f"MAE no teste: {test_mae} \n"
      f"{70*'-'}")

RMSE no teste: 1471.7406104171837 
MAE no teste: 472.4793336721696 
----------------------------------------------------------------------


## Loja 3

### Criando um modelo segmentado para a loja 3


In [ ]:
# definindo uma loja específica
store_nbr = 3

# filtrando os dados de treino e teste para a loja 3
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# definindo features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# separando as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# convertendo as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# definindo os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # armazenando as predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# avaliando os modelos para a loja 3
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# calculando métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

Treinando o algoritmo: LinearRegression para a loja 3
Treino finalizado. Tempo gasto no treinamento: 0.33 segundos
RMSE no treino: 5224.2017644061225 
RMSE no teste: 3399.9756517718006 
MAE no treino: 1799.8472811077115 
MAE no teste: 1496.4603038940934 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 3
Treino finalizado. Tempo gasto no treinamento: 3.77 segundos
RMSE no treino: 2282.312590757182 
RMSE no teste: 4880.978974982836 
MAE no treino: 919.3906704197835 
MAE no teste: 1692.6880798327102 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 3
Treino finalizado. Tempo gasto no treinamento: 7.37 segundos
RMSE no treino: 4292.44126910176 
RMSE no teste: 3532.8653245858263 
MAE no treino: 1485.0786976017014 
MAE no teste: 1398.5666107138113 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting par

### Comparando com o modelo único criado anteriormente

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_gradboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')
preds_gradboost = pd.DataFrame(pipeline_best_model_gradboost.predict(X_test))
preds_gradboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 3
df = pd.concat([X_test,preds_gradboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 3].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_gradboost_loja_3 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 2797.1018338478757
mae = 988.2779217698555


Na loja 3, o modelo único apresentou um desempenho melhor do que qualquer um dos modelos segmentados, porém vamos tentar boostar o modelo de Catboost que pareceu um modelo promissor.

### Tunando os hiperparâmetros do melhor modelo para a loja 3

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros da primeira iteração
initial_params = {
    'iterations': 400,
    'depth': 5,
    'learning_rate': 0.01,
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        iterations = initial_params['iterations']
        depth = initial_params['depth']
        learning_rate = initial_params['learning_rate']

        # Imprimir os valores iniciais
        print(f"Trial 0: iterations={iterations}, depth={depth}, learning_rate={learning_rate}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        iterations = trial.suggest_int('iterations', 200, 2000)
        depth = trial.suggest_int('depth', 2, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = cb.CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=0
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 10 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=10)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=30, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2025-08-21 13:42:47,923] A new study created in memory with name: no-name-5ccc091d-cad4-4c99-9d02-8db443344fcd


Trial 0: iterations=400, depth=5, learning_rate=0.01


[I 2025-08-21 13:42:58,838] Trial 1 finished with value: 3158.431874411851 and parameters: {'iterations': 515, 'depth': 2, 'learning_rate': 0.038947404410239445}. Best is trial 1 with value: 3158.431874411851.
[I 2025-08-21 13:43:03,998] Trial 0 finished with value: 3532.8653245858263 and parameters: {}. Best is trial 1 with value: 3158.431874411851.
[I 2025-08-21 13:44:10,206] Trial 2 finished with value: 3432.1706902694655 and parameters: {'iterations': 1281, 'depth': 6, 'learning_rate': 0.06505588892151334}. Best is trial 1 with value: 3158.431874411851.
[I 2025-08-21 13:44:33,584] Trial 4 finished with value: 3761.526550175196 and parameters: {'iterations': 462, 'depth': 3, 'learning_rate': 0.09982513198035413}. Best is trial 1 with value: 3158.431874411851.
[I 2025-08-21 13:44:52,011] Trial 5 finished with value: 3130.8823189949444 and parameters: {'iterations': 275, 'depth': 5, 'learning_rate': 0.03876617279389965}. Best is trial 5 with value: 3130.8823189949444.
[I 2025-08-21 13

Melhores hiperparâmetros: {'iterations': 551, 'depth': 3, 'learning_rate': 0.014124763094261859}
Melhor RMSE: 2970.5437


In [ ]:
best_params = study.best_params

model = cb.CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=0,
    random_state=0
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja3 = pipeline.predict(X_test_store)

In [ ]:
# Conferindo o desempenho da versão salva

results = []

# Previsões no teste
test_predictions = pipeline.predict(X_test_store)
test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
test_mae = mean_absolute_error(y_test_store, test_predictions)

print(f"RMSE no teste: {test_rmse} \n"
      f"MAE no teste: {test_mae} \n"
      f"{70*'-'}")

RMSE no teste: 2970.54373330122 
MAE no teste: 1129.0080345042263 
----------------------------------------------------------------------


Mesmo com o modelo otimizado ele não foi melhor que o modelo geral, então não precisamos salvar esse modelo.

## Loja 4

### Criando um modelo segmentado para a loja 4

In [ ]:
# definindo uma loja específica
store_nbr = 4

# filtrando os dados de treino e teste para a loja 4
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# definindo features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# separando as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# convertendo as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# definindo os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # armazenando as predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# avaliando os modelos para a loja 4
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# calculando métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

Treinando o algoritmo: LinearRegression para a loja 4
Treino finalizado. Tempo gasto no treinamento: 0.33 segundos
RMSE no treino: 1868.8583972273948 
RMSE no teste: 1442.4641880431157 
MAE no treino: 682.4046242009894 
MAE no teste: 636.3023447691331 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 4
Treino finalizado. Tempo gasto no treinamento: 5.17 segundos
RMSE no treino: 867.4690510245873 
RMSE no teste: 1621.6786173688743 
MAE no treino: 362.9879817929973 
MAE no teste: 567.3745324301257 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 4
Treino finalizado. Tempo gasto no treinamento: 6.45 segundos
RMSE no treino: 1521.6747462025246 
RMSE no teste: 1442.3130586250663 
MAE no treino: 550.6861935936157 
MAE no teste: 550.8848373403031 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para 

### Comparando com o modelo único criado anteriormente

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_gradboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')
preds_gradboost = pd.DataFrame(pipeline_best_model_gradboost.predict(X_test))
preds_gradboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 4
df = pd.concat([X_test,preds_gradboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 4].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_gradboost_loja_4 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 1332.9794359463199
mae = 499.2058301126813


Podemos verificar que o modelo que apresentou o melhor desempenho para a loja 4 foi o modelo único, porém iremos realizar uma tunagem nos hiperparametros do modelo de catboost para verificar se supera o modelo único.

### Tunando os hiperparâmetros do melhor modelo para a loja 4

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir os hiperparâmetros da primeira iteração
initial_params = {
    'iterations': 400,
    'depth': 5,
    'learning_rate': 0.01,
}

# Função objetiva para a otimização do Optuna
def objective(trial):

    # Para a primeira iteração, use os valores iniciais fornecidos
    if trial.number == 0:
        iterations = initial_params['iterations']
        depth = initial_params['depth']
        learning_rate = initial_params['learning_rate']

        # Imprimir os valores iniciais
        print(f"Trial 0: iterations={iterations}, depth={depth}, learning_rate={learning_rate}")

    else:
        # Sugerir valores para os hiperparâmetros a partir da segunda iteração
        iterations = trial.suggest_int('iterations', 200, 2000)
        depth = trial.suggest_int('depth', 3, 9)
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1, log=True)

    # Definir o modelo com os hiperparâmetros sugeridos
    model = cb.CatBoostRegressor(
        iterations=iterations,
        depth=depth,
        learning_rate=learning_rate,
        loss_function='RMSE',
        verbose=0,
        random_state=0
    )

    # Pipeline com o preprocessador e o modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar o pipeline
    pipeline.fit(X_train_store, y_train_store)

    # Prever no conjunto de validação
    y_pred = pipeline.predict(X_test_store)

    # Calcular o erro quadrático médio
    rmse = mean_squared_error(y_test_store, y_pred)**0.5

    return rmse

class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Crie o estudo e defina o callback para parar após 10 iterações sem melhorias
early_stopping = EarlyStoppingCallback(patience=10)

# Criação do estudo Optuna com o sampler TPE
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=50, n_jobs=-1, callbacks=[early_stopping])

# Melhor conjunto de hiperparâmetros encontrado
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor RMSE: {study.best_value:.4f}")

[I 2025-08-21 14:42:49,146] A new study created in memory with name: no-name-13541447-9844-4930-bbdd-dcc999c6d49a


Trial 0: iterations=400, depth=5, learning_rate=0.01


[I 2025-08-21 14:43:14,822] Trial 93 finished with value: 1334.692836453488 and parameters: {'iterations': 1216, 'depth': 6, 'learning_rate': 0.06911535851944071}. Best is trial 76 with value: 1282.2840996308025.
[I 2025-08-21 14:43:30,699] Trial 0 finished with value: 1442.3130586250663 and parameters: {}. Best is trial 0 with value: 1442.3130586250663.
[I 2025-08-21 14:43:48,733] Trial 1 finished with value: 1462.6815853180744 and parameters: {'iterations': 1237, 'depth': 3, 'learning_rate': 0.023602048766730162}. Best is trial 0 with value: 1442.3130586250663.
[I 2025-08-21 14:43:56,123] Trial 94 finished with value: 1391.5834446331544 and parameters: {'iterations': 1081, 'depth': 6, 'learning_rate': 0.07190627044059911}. Best is trial 76 with value: 1282.2840996308025.
[I 2025-08-21 14:44:12,868] Trial 2 finished with value: 1318.917534989469 and parameters: {'iterations': 830, 'depth': 5, 'learning_rate': 0.03679800713056946}. Best is trial 2 with value: 1318.917534989469.
[I 2025

Melhores hiperparâmetros: {'iterations': 830, 'depth': 5, 'learning_rate': 0.03679800713056946}
Melhor RMSE: 1318.9175


In [ ]:
best_params = study.best_params

model = cb.CatBoostRegressor(
    iterations=best_params['iterations'],
    depth=best_params['depth'],
    learning_rate=best_params['learning_rate'],
    loss_function='RMSE',
    verbose=0,
    random_state=0
)

pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                            ('model', model)])
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)
y_pred_loja4 = pipeline.predict(X_test_store)

In [ ]:
# Conferindo o desempenho da versão tunada

results = []

# Previsões no teste
test_predictions = pipeline.predict(X_test_store)
test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
test_mae = mean_absolute_error(y_test_store, test_predictions)

print(f"RMSE no teste: {test_rmse} \n"
      f"MAE no teste: {test_mae} \n"
      f"{70*'-'}")

RMSE no teste: 1318.917534989469 
MAE no teste: 465.1275726655906 
----------------------------------------------------------------------


O modelo tunado de Catboost obteve um valor mais alto que o modelo único para a loja 4, então vamos salva-lo.

In [ ]:
# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja4.pkl')

['/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja4.pkl']

## Loja 5

### Criando um modelo segmentado para a loja 5

In [ ]:
# definindo uma loja específica
store_nbr = 5

# filtrando os dados de treino e teste para a loja 5
train_store = train[train['store_nbr'] == store_nbr]
val_store = val[val['store_nbr'] == store_nbr]

# definindo features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# separando as colunas numéricas e categóricas
numeric_features = train_store[features].select_dtypes(include=['int64', 'float64']).columns.to_list()
categorical_features = train_store[features].select_dtypes(include=['object']).columns.to_list()

# convertendo as colunas categóricas para string
train_store.loc[:,categorical_features] = train_store[categorical_features].astype(str)
val_store.loc[:,categorical_features] = val_store[categorical_features].astype(str)

X_train_store = train_store[numeric_features + categorical_features]
y_train_store = train_store[target]
X_test_store = val_store[numeric_features + categorical_features]
y_test_store = val_store[target]

# preprocessamento para dados numéricos e categóricos
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# definindo os modelos a serem testados
models = {
    'LinearRegression': LinearRegression(),
    'LightGBM': lgb.LGBMRegressor(n_estimators=400, random_state=42, verbose=-1, learning_rate=0.01),
    'CatBoost': cb.CatBoostRegressor(iterations=400, depth=5, learning_rate=0.01, loss_function='RMSE', verbose=0, random_state=0),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=450, random_state=42,learning_rate=0.01),
    'XGBoost': XGBRegressor(n_estimators=400, random_state=42, verbosity=0, learning_rate=0.01)
}

# Função para treinar e avaliar modelos
def evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store):
    results = []
    train_predictions_all = []
    test_predictions_all = []

    for model_name, model in models.items():
        pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                                   ('model', model)])

        print(f'Treinando o algoritmo: {model_name} para a loja {store_nbr}')
        start_time = time.time()
        pipeline.fit(X_train_store, y_train_store)
        end_time = time.time()
        elapsed_time = end_time - start_time
        print(f'Treino finalizado. Tempo gasto no treinamento: {elapsed_time:.2f} segundos')

        # Previsões no treino
        train_predictions = pipeline.predict(X_train_store)
        train_rmse = np.sqrt(mean_squared_error(y_train_store, train_predictions))
        train_mae = mean_absolute_error(y_train_store, train_predictions)

        # Previsões no teste
        test_predictions = pipeline.predict(X_test_store)
        test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
        test_mae = mean_absolute_error(y_test_store, test_predictions)

        results.append((model_name, train_rmse, test_rmse, train_mae, test_mae))
        print(f"RMSE no treino: {train_rmse} \n"
              f"RMSE no teste: {test_rmse} \n"
              f"MAE no treino: {train_mae} \n"
              f"MAE no teste: {test_mae} \n"
              f"{70*'-'}")

        # armazenando as predições
        train_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': train_predictions, 'actual': y_train_store}))
        test_predictions_all.append(pd.DataFrame({'store_nbr': store_nbr, 'model': model_name, 'pred': test_predictions, 'actual': y_test_store}))

    return results, pd.concat(train_predictions_all), pd.concat(test_predictions_all)

# avaliando os modelos para a loja 5
results, train_predictions, test_predictions = evaluate_models(models, X_train_store, y_train_store, X_test_store, y_test_store)

# calculando métricas globais para cada modelo
global_metrics = []
for model_name in test_predictions['model'].unique():
    model_test_predictions = test_predictions[test_predictions['model'] == model_name]
    global_rmse = np.sqrt(mean_squared_error(model_test_predictions['actual'], model_test_predictions['pred']))
    global_mae = mean_absolute_error(model_test_predictions['actual'], model_test_predictions['pred'])
    global_metrics.append((model_name, global_rmse, global_mae))

# Resultados globais
global_metrics_df = pd.DataFrame(global_metrics, columns=['Model', 'Global RMSE', 'Global MAE'])

print('Métricas globais:')
print(global_metrics_df)

Treinando o algoritmo: LinearRegression para a loja 5
Treino finalizado. Tempo gasto no treinamento: 0.40 segundos
RMSE no treino: 1624.9328894112484 
RMSE no teste: 1059.045576760986 
MAE no treino: 605.4084319546157 
MAE no teste: 551.429705502591 
----------------------------------------------------------------------
Treinando o algoritmo: LightGBM para a loja 5
Treino finalizado. Tempo gasto no treinamento: 6.59 segundos
RMSE no treino: 672.5989384310467 
RMSE no teste: 893.1089188855083 
MAE no treino: 296.59227039750317 
MAE no teste: 326.457419361763 
----------------------------------------------------------------------
Treinando o algoritmo: CatBoost para a loja 5
Treino finalizado. Tempo gasto no treinamento: 7.17 segundos
RMSE no treino: 1276.7835791236669 
RMSE no teste: 834.6483015652094 
MAE no treino: 477.7968369776906 
MAE no teste: 340.20097826517053 
----------------------------------------------------------------------
Treinando o algoritmo: GradientBoosting para a l

### Comparando com o modelo único criado anteriormente

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_gradboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')
preds_gradboost = pd.DataFrame(pipeline_best_model_gradboost.predict(X_test))
preds_gradboost.index = X_test.index
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas da Loja 5
df = pd.concat([X_test,preds_gradboost,yreal],axis=1).rename(columns={0:'pred'}).copy()
df1 = df[df.store_nbr == 5].copy()

rmse = np.sqrt(mean_squared_error(df1['target'], df1['pred']))
mae = mean_absolute_error(df1['target'], df1['pred'])

preds_gradboost_loja_5 = np.copy(df1.pred)

print(f'rmse = {rmse}')
print(f'mae = {mae}')

rmse = 931.5850728920358
mae = 359.65121274366226


O modelo segmentado de gradiente boost conseguiu o melhor desempenho entre todos os modelos, então vamos tunar seus hiperparâmetros.

### Tunando os hiperparâmetros do melhor modelo para a loja 5

In [ ]:
# Pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Função objetiva para a otimização do Optuna
def objective(trial):
    # Hiperparâmetros a otimizar (compatíveis com GradientBoostingRegressor)
    n_estimators = trial.suggest_int('n_estimators', 100, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.2, log=True)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)

    # Definir o modelo Gradient Boosting
    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        max_features=max_features,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )

    # Pipeline com pré-processador e modelo
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('model', model)])

    # Treinar
    pipeline.fit(X_train_store, y_train_store)

    # Prever
    y_pred = pipeline.predict(X_test_store)

    # Calcular MSE
    mse = mean_squared_error(y_test_store, y_pred)
    return mse

# Callback de EarlyStopping
class EarlyStoppingCallback:
    def __init__(self, patience: int):
        self.patience = patience
        self.best_score = None
        self.num_no_improvement = 0

    def __call__(self, study: optuna.study.Study, trial: optuna.trial.FrozenTrial):
        current_score = study.best_value
        if self.best_score is None or current_score < self.best_score:
            self.best_score = current_score
            self.num_no_improvement = 0
        else:
            self.num_no_improvement += 1

        if self.num_no_improvement >= self.patience:
            study.stop()

# Early stopping após 10 iterações sem melhoria
early_stopping = EarlyStoppingCallback(patience=10)

# Rodar Optuna
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=42))
study.optimize(objective, n_trials=30, n_jobs=-1, callbacks=[early_stopping])

# Resultados
print(f"Melhores hiperparâmetros: {study.best_params}")
print(f"Melhor MSE: {study.best_value:.4f}")

[I 2025-08-21 16:16:16,908] A new study created in memory with name: no-name-cf4c85da-226a-4e75-b109-e828a3b4a852
[I 2025-08-21 16:17:45,296] Trial 1 finished with value: 726020.050223625 and parameters: {'n_estimators': 324, 'max_depth': 8, 'learning_rate': 0.08312777588846845, 'subsample': 0.9619805574651765, 'max_features': 'sqrt', 'min_samples_split': 8, 'min_samples_leaf': 19}. Best is trial 1 with value: 726020.050223625.
[I 2025-08-21 16:20:05,115] Trial 0 finished with value: 612217.9253590405 and parameters: {'n_estimators': 941, 'max_depth': 10, 'learning_rate': 0.14692293430386028, 'subsample': 0.7664402154291494, 'max_features': 'sqrt', 'min_samples_split': 11, 'min_samples_leaf': 5}. Best is trial 0 with value: 612217.9253590405.
[I 2025-08-21 16:21:39,469] Trial 3 finished with value: 629456.4382324932 and parameters: {'n_estimators': 835, 'max_depth': 8, 'learning_rate': 0.06936283130702803, 'subsample': 0.6181823846451466, 'max_features': 'log2', 'min_samples_split': 5,

Melhores hiperparâmetros: {'n_estimators': 302, 'max_depth': 7, 'learning_rate': 0.08416452868520966, 'subsample': 0.9109661090441825, 'max_features': 'log2', 'min_samples_split': 11, 'min_samples_leaf': 13}
Melhor MSE: 575045.9922


In [ ]:
best_params = study.best_params

# Define o modelo Gradient Boosting com os parâmetros encontrados
model = GradientBoostingRegressor(
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    max_features=best_params['max_features'],
    min_samples_split=best_params['min_samples_split'],
    min_samples_leaf=best_params['min_samples_leaf'],
    random_state=42
)

# Pipeline com pré-processador e modelo
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Treinar e medir tempo
start_time = time.time()
pipeline.fit(X_train_store, y_train_store)

# Prever
y_pred_loja5 = pipeline.predict(X_test_store)

# Tempo de treino
print(f"Tempo de treino: {time.time() - start_time:.2f} segundos")

Tempo de treino: 38.33 segundos


In [ ]:
# Conferindo o desempenho da versão tunada

results = []

# Previsões no teste
test_predictions = pipeline.predict(X_test_store)
test_rmse = np.sqrt(mean_squared_error(y_test_store, test_predictions))
test_mae = mean_absolute_error(y_test_store, test_predictions)

print(f"RMSE no teste: {test_rmse} \n"
      f"MAE no teste: {test_mae} \n"
      f"{70*'-'}")

RMSE no teste: 758.3178701346526 
MAE no teste: 255.4356954776338 
----------------------------------------------------------------------


In [ ]:
# Salvando o pipeline treinado como um arquivo .pkl
joblib.dump(pipeline, '/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja5.pkl')

['/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_modelo_segmentado_loja5.pkl']

## Avaliação dos resultados no público completo

### Avaliação técnica do modelo único

In [ ]:
# Importando pipeline e realizando as predições no dataset de test
pipeline_best_model_catboost = joblib.load('/content/drive/MyDrive/2 - Projetos Portfolio/Previsão de demanda/Base de dados/pipeline_best_model_all_features_gradient_boosting.pkl')

# Definir features e target
features = [col for col in train.columns if col not in ['id','date','target']]
target = 'target'

# Separar as colunas numéricas e categóricas
numeric_features = train[features].select_dtypes(include=['int64', 'float64']).columns
categorical_features = train[features].select_dtypes(include=['object']).columns

train.loc[:,categorical_features] = train[categorical_features].astype(str)
val.loc[:,categorical_features] = val[categorical_features].astype(str)

X_train = train[features]
y_train = train[target]
X_test = val[features]
y_test = val[target]

preds_catboost = pd.DataFrame(pipeline_best_model_catboost.predict(X_test))
yreal = pd.DataFrame(y_test)

# Avaliando a performance desse modelo em prever as demandas de todas as lojas
df = pd.concat([X_test,preds_catboost,yreal],axis=1).rename(columns={0:'pred'})

rmse_modelo_unico = np.sqrt(mean_squared_error(df['target'], df['pred']))
mae_modelo_unico = mean_absolute_error(df['target'], df['pred'])

print(f'RMSE = {rmse_modelo_unico}')
print(f'mae = {mae_modelo_unico}')

RMSE = 1707.5415346901768
mae = 565.0829515040268


### 4.2. Avaliação técnica dos modelos segmentados

In [ ]:
result = np.hstack((y_pred_loja1, y_pred_loja2, preds_gradboost_loja_3, y_pred_loja4, y_pred_loja5))

rmse_modelo_segmentado = np.sqrt(mean_squared_error(yreal, result))
mae_modelo_segmentado = mean_absolute_error(yreal, result)

print(f'rmse = {rmse_modelo_segmentado}')
print(f'mae = {mae_modelo_segmentado}')

rmse = 3321.1877641488277
mae = 939.060433784871


In [ ]:
df_metrics = pd.DataFrame({
    'Modelo': ['Único', 'Segmentado'],
    'RMSE': [rmse_modelo_unico, rmse_modelo_segmentado],
    'MAE': [mae_modelo_unico, mae_modelo_segmentado]
})

df_metrics

,Modelo,RMSE,MAE
0,Único,1707.541535,565.082952
1,Segmentado,3321.187764,939.060434


No quesito geral, o modelo único tem um desempenho muito superior do que os modelos segmentados, então ele pode ser utilizado para todas as lojas em geral ou utilizamos somente nas lojas que ele apresentou um melhor resultado, depende da estratégia da empresa.